# Advanced Recursion Patterns in UnifyWeaver

This notebook demonstrates the four main recursion patterns that UnifyWeaver can detect and optimize:

1. **Tail Recursion** - Iterative loops with accumulators
2. **Linear Recursion** - Single recursive call with memoization
3. **Tree Recursion** - Multiple recursive calls on structure parts
4. **Mutual Recursion** - Predicates calling each other in cycles

## Learning Objectives

- Understand different recursion patterns
- See how UnifyWeaver detects and optimizes each pattern
- Compare performance characteristics
- Learn when to use each pattern

## Setup

Initialize the UnifyWeaver environment.

In [ ]:
% Load initialization
['../init'].

% Load necessary modules
use_module(unifyweaver(core/recursive_compiler)).
use_module(unifyweaver(core/advanced/pattern_matchers)).

## Pattern 1: Tail Recursion

Tail recursion uses an accumulator to carry intermediate results, and the recursive call is the **last action** in the function.

### Example: Count Items in a List

In [ ]:
% Define tail-recursive count_items
:- dynamic count_items/3.

% Base case: empty list, return accumulator
count_items([], Acc, Acc).

% Recursive case: increment accumulator, recurse on tail
count_items([_|T], Acc, N) :-
    Acc1 is Acc + 1,
    count_items(T, Acc1, N).  % ← Tail position!

### Test in Prolog

In [ ]:
% Test: count items in [a,b,c,d,e]
count_items([a,b,c,d,e], 0, N),
format('Count: ~w~n', [N]).

### Check Pattern Detection

In [ ]:
% Check if detected as tail recursive
is_tail_recursive_accumulator(count_items/3, AccInfo),
format('Tail recursive: ~w~n', [AccInfo]).

### Compile to Bash

In [ ]:
% Compile and save
compile_recursive(count_items/3, [], BashCode),
open('../output/count_items_demo.sh', write, Stream),
write(Stream, BashCode),
close(Stream),
writeln('✓ Compiled count_items to Bash with tail recursion optimization').

### Test Generated Bash

In [ ]:
%%bash
source ../output/count_items_demo.sh
echo "Counting items in [a,b,c,d,e]:"
count_items "[a,b,c,d,e]" 0 ""

## Pattern 2: Linear Recursion

Linear recursion has **exactly one** recursive call per clause, with computation happening after the recursive call returns.

### Example: Factorial

In [ ]:
% Define factorial
:- dynamic factorial/2.

% Base case
factorial(0, 1).

% Recursive case: exactly ONE recursive call
factorial(N, F) :-
    N > 0,
    N1 is N - 1,
    factorial(N1, F1),  % ← One recursive call
    F is N * F1.        % ← Computation after call

### Test in Prolog

In [ ]:
% Test: factorial of 5
factorial(5, F),
format('5! = ~w~n', [F]).

### Check Pattern Detection

In [ ]:
% Check if detected as linear recursive
is_linear_recursive_streamable(factorial/2),
writeln('✓ Detected as linear recursion').

### Compile to Bash

In [ ]:
% Compile and save
compile_recursive(factorial/2, [], BashCode),
open('../output/factorial_demo.sh', write, Stream),
write(Stream, BashCode),
close(Stream),
writeln('✓ Compiled factorial to Bash with fold-based linear recursion').

### Test Generated Bash

In [ ]:
%%bash
source ../output/factorial_demo.sh
echo "Factorial of 5:"
factorial 5 ""
echo ""
echo "Factorial of 10:"
factorial 10 ""

## Pattern 3: Tree Recursion

Tree recursion makes **multiple** recursive calls to process different parts of a structure.

### Example: Tree Sum

In [ ]:
% Define tree_sum for binary trees
% Tree format: [Value, LeftSubtree, RightSubtree] or []
:- dynamic tree_sum/2.

% Base case: empty tree has sum 0
tree_sum([], 0).

% Recursive case: sum = value + left_sum + right_sum
tree_sum([V, L, R], Sum) :-
    tree_sum(L, LS),   % ← First recursive call
    tree_sum(R, RS),   % ← Second recursive call
    Sum is V + LS + RS.

### Test in Prolog

In [ ]:
% Test: tree_sum of [5, [3, [1, [], []], []], [2, [], []]]
%       5
%      / \
%     3   2
%    /
%   1
tree_sum([5, [3, [1, [], []], []], [2, [], []]], Sum),
format('Tree sum: ~w (expected 11)~n', [Sum]).

### Compile to Bash

In [ ]:
% Compile and save
compile_recursive(tree_sum/2, [], BashCode),
open('../output/tree_sum_demo.sh', write, Stream),
write(Stream, BashCode),
close(Stream),
writeln('✓ Compiled tree_sum to Bash with tree recursion').

### Test Generated Bash

In [ ]:
%%bash
source ../output/tree_sum_demo.sh
echo "Tree sum of [5,[3,[1,[],[]]],[2,[]]]:"
tree_sum "[5,[3,[1,[],[]]],[2,[],[]]]

## Pattern 4: Mutual Recursion

Mutual recursion occurs when two or more predicates call each other in a cycle.

### Example: Even and Odd

In [ ]:
% Define mutually recursive is_even and is_odd
:- dynamic is_even/1.
:- dynamic is_odd/1.

% is_even base case
is_even(0).

% is_even recursive: N is even if N-1 is odd
is_even(N) :-
    N > 0,
    N1 is N - 1,
    is_odd(N1).  % ← Calls is_odd

% is_odd base case
is_odd(1).

% is_odd recursive: N is odd if N-1 is even
is_odd(N) :-
    N > 1,
    N1 is N - 1,
    is_even(N1).  % ← Calls is_even

### Test in Prolog

In [ ]:
% Test even/odd
is_even(0), writeln('✓ 0 is even').
is_even(4), writeln('✓ 4 is even').
is_odd(3), writeln('✓ 3 is odd').
is_odd(7), writeln('✓ 7 is odd').

### Check for Mutual Recursion

In [ ]:
% Build call graph and find SCCs
use_module(unifyweaver(core/advanced/call_graph)),
use_module(unifyweaver(core/advanced/scc_detection)),

build_call_graph([is_even/1, is_odd/1], Graph),
format('Call graph: ~w~n', [Graph]),

find_sccs(Graph, SCCs),
format('SCCs (mutual recursion groups): ~w~n', [SCCs]).

### Compile to Bash

In [ ]:
% Compile the mutual recursion group
use_module(unifyweaver(core/advanced/mutual_recursion)),

compile_mutual_recursion([is_even/1, is_odd/1], [], BashCode),
open('../output/even_odd_demo.sh', write, Stream),
write(Stream, BashCode),
close(Stream),
writeln('✓ Compiled is_even/is_odd to Bash with shared memoization').

### Test Generated Bash

In [ ]:
%%bash
source ../output/even_odd_demo.sh

echo "Testing is_even and is_odd:"
is_even 0 && echo "✓ 0 is even"
is_even 4 && echo "✓ 4 is even"
is_odd 3 && echo "✓ 3 is odd"
is_odd 7 && echo "✓ 7 is odd"
is_even 5 || echo "✓ 5 is not even"

## Pattern Comparison

Let's compare the characteristics of each pattern:

| Pattern | Recursive Calls | Optimization | Space Complexity | Best For |
|:--------|:----------------|:-------------|:-----------------|:---------|
| **Tail** | 1 (tail position) | Iterative loop | O(1) | Accumulators, linear scans |
| **Linear** | 1 (any position) | Fold + memoization | O(n) memo table | Fibonacci, factorial |
| **Tree** | 2+ (structure parts) | Structural decomposition | O(depth) stack | Tree/graph operations |
| **Mutual** | 1+ (across predicates) | Shared memoization | O(n) shared table | Even/odd, mutual definitions |

## Pattern Detection Order

UnifyWeaver tries patterns in this order:

1. **Tail Recursion** (most efficient)
2. **Linear Recursion** (unless forbidden)
3. **Tree Recursion** (structural)
4. **Mutual Recursion** (SCC detection)
5. **Basic Recursion** (fallback)

You can influence detection with `forbid_linear_recursion/1`.

## Exercise: Your Turn!

Try defining and compiling these predicates:

### 1. Tail Recursive Sum
```prolog
sum_list([], Acc, Acc).
sum_list([H|T], Acc, Sum) :-
    Acc1 is Acc + H,
    sum_list(T, Acc1, Sum).
```

### 2. Linear Recursive Fibonacci
```prolog
fib(0, 0).
fib(1, 1).
fib(N, F) :-
    N > 1,
    N1 is N - 1,
    N2 is N - 2,
    fib(N1, F1),
    fib(N2, F2),
    F is F1 + F2.
```

### 3. Tree Height
```prolog
tree_height([], 0).
tree_height([_, L, R], H) :-
    tree_height(L, HL),
    tree_height(R, HR),
    H is max(HL, HR) + 1.
```

In [ ]:
% Your code here!


## Summary

In this notebook, you learned:

✅ The four main recursion patterns in UnifyWeaver

✅ How to define each pattern in Prolog

✅ How UnifyWeaver detects and optimizes each pattern

✅ The performance characteristics of each pattern

✅ When to use each pattern

## Next Steps

Continue to **Notebook 3: Call Graph Visualization** to learn about advanced code analysis and visualization!